In [5]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [6]:
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [7]:
train, test = train_test_split(data, test_size=0.25, random_state=42)

In [8]:
train_x = train.drop(['quality'], axis=1).values
train_y = train[['quality']].values.ravel()

test_x = test.drop(['quality'], axis=1).values
test_y = test[['quality']].values.ravel()

train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [9]:
signature = infer_signature(train_x, train_y)

In [10]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model = keras.Sequential([
        keras.Input([train_x.shape[1]]),
        keras.layers.Normalization(mean=mean, variance=var),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1)
    ])

    model.compile(opimizer=keras.optimizers.SGD(learning_rate=params['lr'], momentum=params['momentum']),
                  loss='mean_squared_error',
                  metrics=[keras.metrics.RootMeanSquaredError()])
    
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, epochs=epochs, batch_size=64, validation_data=(valid_x, valid_y))

        eval_result = model.evaluate(test_x, test_y, batch_size=64)
        eval_rmse = eval_result[1]

        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [12]:
def objective(params):
    result = train_model(params, 3, train_x, train_y, valid_x, valid_y, test_x, test_y)
    return result